# Radiometer comparison

This notebook serves to read in the .dat file with 1 minute measurements for both net radiometers, get some basic statistics and make plots for comparison.

pyranometer = SW  
pyrgeometer = LW

CNR4:  
https://www.kippzonen.com/Product/85/CNR4-Net-Radiometer


SN500:  
https://www.apogeeinstruments.com/sn-500-ss-net-radiometer/  
https://www.apogeeinstruments.com/content/SN-500-manual.pdf  
https://www.apogeeinstruments.com/content/SN-500-spec-sheet.pdf

## Settings

Here it is possible to set whether the data should be cut off at a given index and whether only midday values should be shown (and between which times of the day). 
These settings are later passed to the plotting functions as keyword arguments.  
Of course it is also possible to pass different arguments to each function, however, these setting are practical if one wants to make all the plots with the same settings.

In [ ]:
# select only data up to a given index
cut = True
cut_index = 150000

# select if only data from +/- 1 hour around noon should be used (or whatever times are chosen)
only_midday = False
start_time = '11:00'
end_time = '13:00'

## Imports & preparation of data

In [ ]:
# imports
import numpy as np
import pandas as pd

# imports for non-interactive plots
import matplotlib as mpl
import matplotlib.pyplot as plt
# cyclic colormap for time of day: if cmasher is not installed, use an uglier palette from matplotblib
# but seriously, get cmasher, it has nice colormaps!
try:
    import cmasher as cmr
    day_palette = cmr.infinity
except ModuleNotFoundError:
    day_palette = 'twilight_shifted'

# imports for interactive plots
from bokeh.plotting import figure, show
from bokeh.layouts import column, gridplot, row
from bokeh.models.sources import ColumnDataSource
from bokeh.models.tools import HoverTool
from bokeh.models import Range1d, Span, Legend, LinearAxis
from bokeh.io import output_notebook


# show interactive bokeh plots in the notebook
output_notebook()

# path to data
file = 'CR1000XSeries_RadComparisionRoofTop_Table1_OverlappingPeriod.dat'

In [ ]:
# Load and pre-process the data, calculate extra variables

# for exploration: get info from the file headers: units and type of data stored (avg/std)
df_info = pd.read_csv(file, skiprows=[0], nrows=2)

# select index + columns with 1 min radiation averages from both instruments
df = pd.read_csv(file,
                 skiprows=[0, 1, 2, 3],
                 usecols=[0, 3, 4, 5, 6, 8, 9, 10, 11],
                 names = ['time',
                          'SWin_CNR4', 'SWout_CNR4',
                          'LWin_CNR4', 'LWout_CNR4',
                          'SWin_SN500', 'SWout_SN500',
                          'LWin_SN500', 'LWout_SN500'],
                 index_col='time',
                 parse_dates=True)

# Calculate extra variables for both datasets 
# I never actually look at any of these in this notebook, but they're there if you're interested!
# Net energy balance
df['Net_balance_CNR4'] = df['SWin_CNR4'] + df['LWin_CNR4'] - \
                         df['SWout_CNR4'] - df['LWout_CNR4']
df['Net_balance_SN500'] = df['SWin_SN500'] + df['LWin_SN500'] - \
                          df['SWout_SN500'] - df['LWout_SN500']
# Net shortwave radiation
df['SWnet_CNR4'] = df['SWin_CNR4'] - df['SWout_CNR4']
df['SWnet_SN500'] = df['SWin_SN500'] - df['SWout_SN500']

## Plotting functions: non-interactive

In [ ]:
# Function to plot the whole time series: non-interactive
# If cut=True, this funciton still plots the whole time series, and only highlights where the data will be cut

def plot_full_timeseries_noninteractive(df, var, markersize=1, 
                                        cut=True, cut_index=150000,
                                        only_midday=False, start_time='11:00', end_time='13:00'):
    # choose a given variable for both instruments from the full data set
    var_cnr = df[var + '_CNR4'].copy()
    var_sn = df[var + '_SN500'].copy()
    
    if only_midday:
        # choose time +/- 1 hour around noon or whatever times are set
        var_cnr = var_cnr.between_time(start_time, end_time)
        var_sn = var_sn.between_time(start_time, end_time)

    # initiate figure
    fig, ax = plt.subplots(figsize=[15, 4])


    # Plot variables: scatter plots
    var_cnr[var_cnr > var_sn].plot(ax=ax,
                                   label='CNR4 (when CNR4 > SN500)',
                                   color='navy',
                                   linestyle='', marker='.',
                                   markersize=markersize)
    var_sn[var_cnr > var_sn].plot(ax=ax,
                                  label='SN500 (when CNR4 > SN500)',
                                  color='darkcyan',
                                  linestyle='', marker='.',
                                  markersize=markersize)

    var_sn[var_sn > var_cnr].plot(ax=ax,
                                  label='SN500 (when SN500 > CNR4)',
                                  color='maroon',
                                  linestyle='', marker='.',
                                  markersize=markersize)
    var_cnr[var_sn > var_cnr].plot(ax=ax,
                                   label='CNR4 (when SN500 > CNR4)',
                                   color='darkorange',
                                   linestyle='', marker='.',
                                   markersize=markersize)

    # Set labels
    ax.set_ylabel('Wm$^{-2}$')
    ax.set_title('Full time series: {}'.format(var), fontsize=16)

    # Axis settings and lines to guide the eye
    ax.autoscale(enable=True, axis='x', tight=True)
    ax.axhline(y=0, linestyle='--', c='k', lw='2')
    
    # always plot full time series, but if data is cut, show where the cut-off is
    if cut:
        ax.axvline(x=var_cnr.index[cut_index], linestyle='-',
                        c='gray', lw='0.5')

    # Legend
    ax.legend(fontsize=14, markerscale=5, facecolor='white', framealpha=1,
              loc=3, bbox_to_anchor=[0.70, 0.65])
        
    return fig, ax

In [ ]:
# Function to make 1:1 plots for each variable

def plot_one_to_one_noninteractive(df, var, 
                                   cut=True, cut_index=150000,
                                   only_midday=False, start_time='11:00', end_time='13:00'):
    
    # choose a given variable for both instruments from the full data set
    var_cnr = var + '_CNR4'
    var_sn = var + '_SN500'
    
    # subset of the full dataset with only the desired variables
    df_subset = df[[var_cnr, var_sn]].copy()
    
    # cut first, then select midday values 
    # (if both "cut" and "only_noon" are True, we want to exclude the noon times past the cut_index as well)
    if cut:
        # select only data up to a given index
        df_subset = df_subset[:cut_index]
    if only_midday:
        # choose time +/- 1 hour around noon or whatever times are set
        df_subset = df_subset.between_time(start_time, end_time)
        
    # initiate figure
    fig, ax = plt.subplots(figsize=[6, 5])

    # plot points
    points = ax.scatter(df_subset[var_cnr], df_subset[var_sn], s=0.2,
                        c=df_subset.index.hour + df_subset.index.minute/60,
                        cmap=day_palette)
    
    # get axis limits based on min/max values
    xy_min = min(df_subset[var_cnr].min(), df_subset[var_sn].min()) * 1.05
    xy_max = min(df_subset[var_cnr].max(), df_subset[var_sn].max()) * 1.05
    
    # Axis settings and labels
    ax.set_title('SN500 vs. CNR4: {}'.format(var), fontsize=18)
    ax.set_xlabel('CNR4 [Wm$^{-2}$]')
    ax.set_ylabel('SN500 [Wm$^{-2}$]')
    ax.set_xlim(xy_min, xy_max)
    ax.set_ylim(xy_min, xy_max)
    


    # Plot reference lines: diagonal and at zero
    ax.plot([xy_min, xy_max], [xy_min, xy_max],
                 linestyle='--', color='k', lw=1.5)
    ax.axvline(x=0, linestyle='--', c='k', lw=1.5)
    ax.axhline(y=0, linestyle='--', c='k', lw=1.5)

    fig.tight_layout()

    # Colorbar settings
    cax = fig.add_axes([0.15, -0.01, 0.85, 0.015])
    cb = fig.colorbar(points, cax=cax,
                      orientation='horizontal')
    cb.set_label('Time of day', fontsize=14)
    
    return fig, ax

## Plotting functions: interactive

In [ ]:
# Function to plot the whole time series: interactive

def plot_full_timeseries(df, var, 
                         plot_width=500, plot_height=300,
                         markersize=0.5, radius=0.5,
                         cut=True, cut_index=150000,
                         only_midday=False, start_time='11:00', end_time='13:00'):
    # choose a given variable for both instruments from the full data set
    var_cnr = var + '_CNR4'
    var_sn = var + '_SN500'
    
    # subset of the full dataset with only the desired variables
    df_subset = df[[var_cnr, var_sn]].copy()
    
    # cut first, then select midday values 
    # (if both "cut" and "only_noon" are True, we want to exclude the noon times past the cut_index as well)
    if cut:
        # select only data up to a given index
        df_subset = df_subset[:cut_index]
    if only_midday:
        # choose time +/- 1 hour around noon or whatever times are set
        df_subset = df_subset.between_time(start_time, end_time)
    
    # set tooltips for hover plots
    tooltips_series = [('Time', '@time{%Y-%m-%d %H:%M}'), 
                       ('CNR4', '@{}'.format(var_cnr)),
                       ('SN500', '@{}'.format(var_sn))]
    # initiate subplot_series for a given variable: timeseries
    subplot_series = figure(tools = 'box_zoom, pan, reset, save',
                            title='Simple time series',
                            plot_width=plot_width, 
                            plot_height=plot_height,
                            x_axis_type='datetime',
                            x_axis_label='Time',
                            y_axis_label='{} '.format(var)+'[W/m^2]')
    
    # Add points at times when CNR4 > SN500
    cds_1 = ColumnDataSource(df_subset[df_subset[var_cnr] > df_subset[var_sn]])
    plt_cnr_1 = subplot_series.circle(x='time', y=var_cnr,
                                      source=cds_1,
                                      hit_dilation=radius,
                                      color='navy',
                                      size=markersize)
    plt_sn_1 = subplot_series.circle(x='time', y=var_sn,
                                     source=cds_1,
                                     hit_dilation=radius,
                                     color='darkcyan',
                                     size=markersize)
    
    # add hover tool for these data points
    h_1 = HoverTool(tooltips=tooltips_series,
                    renderers=[plt_cnr_1], 
                    formatters={"@time": "datetime"},
                    mode='vline')
    # add hover tool for these data points
    subplot_series.add_tools(h_1)
    
    # Add points at times when CNR4 < SN500
    cds_2 = ColumnDataSource(df_subset[df_subset[var_cnr] < df_subset[var_sn]])
    plt_sn_2 = subplot_series.circle(x='time', y=var_sn,
                                     source=cds_2,
                                     hit_dilation=radius,
                                     color='maroon',
                                     size=markersize)
    plt_cnr_2 = subplot_series.circle(x='time', y=var_cnr,
                                      source=cds_2,
                                      hit_dilation=radius,
                                      color='darkorange',
                                      size=markersize)

    # make a custom hover tool that displays only the needed lines
    h_2 = HoverTool(tooltips=tooltips_series,
                    renderers=[plt_sn_2], 
                    formatters={"@time": "datetime"},
                    mode='vline')
    # add hover tool for these data points
    subplot_series.add_tools(h_2)
    
    # set x, y-limits
    subplot_series.x_range = Range1d(df_subset.index[0], df_subset.index[-1])
    
    # horizontal line to guide the eye at 0
    hline = Span(location=0, dimension='width',
                 line_dash='dashed', line_color='black', line_width=1)
    subplot_series.add_layout(hline)
    
    # add legend
    legend = Legend(items=[('SN500 (SN500 > CNR4)', [plt_sn_2]),
                           ('CNR4 (SN500 > CNR4)', [plt_cnr_2]),
                           ('CNR4 (CNR4 > SN500)', [plt_cnr_1]),
                           ('SN500 (CNR4 > SN500)', [plt_sn_1])],
                    location=(20, 20),
                    orientation='horizontal')
    subplot_series.add_layout(legend, 'below')
    
    return subplot_series

In [ ]:
# Function to plot timeseries of the ratio SN500/CNR4 for a given variable

def plot_ratio_timeseries(df, var, plot_width=500, plot_height=300, markersize=0.5, radius=0.5,
                          cut=True, cut_index=150000,
                          only_midday=False, start_time='11:00', end_time='13:00'):
    # choose a given variable for both instruments from the full data set
    var_cnr = var + '_CNR4'
    var_sn = var + '_SN500'
    
    # subset of the full dataset with only the desired variables
    df_subset = df[[var_cnr, var_sn]].copy()
    # get a ratio of the variables
    df_subset['ratio'] = df_subset[var_sn] / df_subset[var_cnr]
    
    # cut first, then select midday values 
    # (if both "cut" and "only_noon" are True, we want to exclude the noon times past the cut_index as well)
    if cut:
        # select only data up to a given index
        df_subset = df_subset[:cut_index]
    if only_midday:
        # choose time +/- 1 hour around noon or whatever times are set
        df_subset = df_subset.between_time(start_time, end_time)
    
    # set tooltips for hover plots
    tooltips = [('Time', '@time{%Y-%m-%d %H:%M}'), 
                ('SN500/CNR4', '@ratio'),
                ('CNR4', '@{}'.format(var_cnr)),
                ('SN500', '@{}'.format(var_sn))]
    # initiate subplot_series for a given variable: timeseries
    subplot = figure(tools = 'box_zoom, pan, reset, save',
                            title='Time series: SN500/CNR4 ratio',
                            plot_width=plot_width, 
                            plot_height=plot_height,
                            x_axis_type='datetime',
                            x_axis_label='Time',
                            y_axis_label='{} ratio'.format(var))
    
    # Plot
    # Add points at times when CNR4 <= 1 (small value in denominator): red
    cds_1 = ColumnDataSource(df_subset[abs(df_subset[var_cnr]) <= 1])
    plt_scatter_1 = subplot.circle(x='time', y='ratio',
                                   source=cds_1,
                                   hit_dilation=radius,
                                   color='red',
                                   size=markersize)
    # add hover tool for these data points
    h_1 = HoverTool(tooltips=tooltips,
                    renderers=[plt_scatter_1], 
                    formatters={"@time": "datetime"},
                    mode='vline')
    # add hover tool for these data points
    subplot.add_tools(h_1)
    
    # Add points at times when CNR4 > 1 (higher value in denominator): blue
    cds_2 = ColumnDataSource(df_subset[abs(df_subset[var_cnr]) > 1])
    plt_scatter_2 = subplot.circle(x='time', y='ratio',
                                   source=cds_2,
                                   hit_dilation=radius,
                                   size=markersize)

    # make a custom hover tool that displays only the needed lines
    h_2 = HoverTool(tooltips=tooltips,
                    renderers=[plt_scatter_2], 
                    formatters={"@time": "datetime"},
                    mode='vline')
    # add hover tool for these data points
    subplot.add_tools(h_2)
    
    
    # set x, y-limits (x tight, y based on percentiles to exclude outliers)
    subplot.x_range = Range1d(df_subset.index[0], df_subset.index[-1])
    subplot.y_range = Range1d(df_subset.ratio.quantile(0.001),
                              df_subset.ratio.quantile(0.999))
    
    # line to guide the eye at 1
    hline = Span(location=1, dimension='width',
                 line_dash='dashed', line_color='black', line_width=1)
    subplot.add_layout(hline)
    
    # add legend
    legend = Legend(items=[('SN500/CNR4 (|CNR4| > 1)', [plt_scatter_2]),
                           ('SN500/CNR4 (|CNR4| <= 1)', [plt_scatter_1])],
                    location=(20, 20),
                    orientation='horizontal')
    subplot.add_layout(legend, 'below')
    
    return subplot

In [ ]:
# Function to plot a difference of the timeseries

def plot_diff_timeseries(df, var, plot_width=500, plot_height=300, markersize=0.5, radius=0.5,
                         cut=True, cut_index=150000,
                         only_midday=False, start_time='11:00', end_time='13:00'):
    # choose a given variable for both instruments from the full data set
    var_cnr = var + '_CNR4'
    var_sn = var + '_SN500'
    
    # subset of the full dataset with only the desired variables
    df_subset = df[[var_cnr, var_sn]].copy()
    # get a ratio of the variables
    df_subset['difference'] = df_subset[var_sn] - df_subset[var_cnr]
    
    # cut first, then select midday values 
    # (if both "cut" and "only_noon" are True, we want to exclude the noon times past the cut_index as well)
    if cut:
        # select only data up to a given index
        df_subset = df_subset[:cut_index]
    if only_midday:
        # choose time +/- 1 hour around noon or whatever times are set
        df_subset = df_subset.between_time(start_time, end_time)
    
    # set tooltips for hover plots
    tooltips = [('Time', '@time{%Y-%m-%d %H:%M}'), 
                ('CNR4', '@{}'.format(var_cnr)),
                ('SN500', '@{}'.format(var_sn))]
    # initiate subplot_series for a given variable: timeseries
    subplot = figure(tools = 'box_zoom, pan, reset, save',
                            title='Time series: SN500 - CNR4 difference',
                            plot_width=plot_width, 
                            plot_height=plot_height,
                            x_axis_type='datetime',
                            x_axis_label='Time',
                            y_axis_label='{} '.format(var)+'[W/m^2]')
    
    # Plot
    cds = ColumnDataSource(df_subset)
    plt_scatter = subplot.circle(x='time', y='difference',
                                 source=cds,
                                 hit_dilation=radius,
                                 size=markersize)
    # add hover tool for these data points
    h = HoverTool(tooltips=tooltips,
                    renderers=[plt_scatter], 
                    formatters={"@time": "datetime"},
                    mode='vline')
    # add hover tool for these data points
    subplot.add_tools(h)
    
    
    # set x, y-limits (x tight, y based on percentiles to exclude outliers)
    subplot.x_range = Range1d(df.index[0], df.index[-1])
    subplot.y_range = Range1d(df_subset.difference.quantile(0.0005),
                              df_subset.difference.quantile(0.9995))
    
    # line to guide the eye at 0
    hline = Span(location=0, dimension='width',
                 line_dash='dashed', line_color='black', line_width=1)
    subplot.add_layout(hline)
    
    return subplot

In [ ]:
# Function to plot the diurnal cycle of a given variable

def plot_diurnal_cycle(df, var, plot_width=500, plot_height=300, line_width=2,
                       cut=True, cut_index=150000,
                       only_midday=False, start_time='11:00', end_time='13:00'):   
    # choose a given variable for both instruments from the full data set
    var_cnr = var + '_CNR4'
    var_sn = var + '_SN500'
    
    # subset of the full dataset with only the desired variables
    df_subset = df[[var_cnr, var_sn]].copy()
    
    # cut first, then select midday values 
    # (if both "cut" and "only_noon" are True, we want to exclude the noon times past the cut_index as well)
    if cut:
        # select only data up to a given index
        df_subset = df_subset[:cut_index]
    if only_midday:
        # choose time +/- 1 hour around noon or whatever times are set
        df_subset = df_subset.between_time(start_time, end_time)

    # get diurnal cycle
    df_cycle = df_subset.groupby(df_subset.index.time).median()
    df_cycle.index.names = ['time']
    # get a ratio of the medians
    df_cycle['ratio'] = df_cycle[var_sn] / df_cycle[var_cnr]

    # set tooltips for hover plots
    tooltips_cycle = [('Time', '@time{%H:%M}'), 
                      ('CNR4', '@{}'.format(var_cnr)),
                      ('SN500', '@{}'.format(var_sn)),
                      ('SN500/CNR4', '@ratio')]
    
    # initiate subplot for a given variable: diurnal cycle
    subplot_cycle = figure(tools = 'box_zoom, pan, reset, save',
                           title='Diurnal cycle (median values)',
                           plot_width=plot_width, 
                           plot_height=plot_height,
                           x_axis_type='datetime',
                           x_axis_label='Time',
                           y_axis_label='{} '.format(var)+'[W/m^2]')
    
    # specify data source, make plots
    cds_c = ColumnDataSource(df_cycle)
    plt_cnr_c = subplot_cycle.line(x='time', y=var_cnr,
                                   source=cds_c,
                                   color='navy',
                                   line_width=line_width)
    plt_sn_c = subplot_cycle.line(x='time', y=var_sn,
                                  source=cds_c,
                                  color='darkcyan',
                                  line_width=line_width)
    
    # add hover tool for these data points
    h_c = HoverTool(tooltips=tooltips_cycle,
                    renderers=[plt_sn_c], 
                    formatters={"@time": "datetime"},
                    mode='vline')
    # add hover tool for these data points
    subplot_cycle.add_tools(h_c)
    
    # set x, y-limits
    subplot_cycle.x_range = Range1d(df_cycle.index[0], df_cycle.index[-1])
    
        # add legend
    legend = Legend(items=[('CNR4', [plt_cnr_c]),
                           ('SN500', [plt_sn_c])],
                    location=(20, 20),
                    orientation='horizontal')
    subplot_cycle.add_layout(legend, 'below')
    
    return subplot_cycle

In [ ]:
# Function to plot a 1:1 scatter plot of a given variable

def plot_one_to_one(df, var, plot_width=500, plot_height=300, markersize=1, radius=3,
                    cut=True, cut_index=150000,
                    only_midday=False, start_time='11:00', end_time='13:00'):   
    # choose a given variable for both instruments from the full data set
    var_cnr = var + '_CNR4'
    var_sn = var + '_SN500'
    
    # subset of the full dataset with only the desired variables
    df_subset = df[[var_cnr, var_sn]].copy()
    
    # cut first, then select midday values 
    # (if both "cut" and "only_noon" are True, we want to exclude the noon times past the cut_index as well)
    if cut:
        # select only data up to a given index
        df_subset = df_subset[:cut_index]
    if only_midday:
        # choose time +/- 1 hour around noon or whatever times are set
        df_subset = df_subset.between_time(start_time, end_time)
    
    # set tooltips for hover plots
    tooltips = [('Time', '@time{%Y-%m-%d %H:%M}'), 
                ('CNR4', '@{}'.format(var_cnr)),
                ('SN500', '@{}'.format(var_sn))]
    
    # initiate subplot for a given variable: diurnal cycle
    subplot = figure(tools = 'box_zoom, pan, reset, save',
                     title='1:1 scatter plot',
                     plot_width=plot_width, 
                     plot_height=plot_height,
                     x_axis_label='{} '.format(var_cnr)+'[W/m^2]',
                     y_axis_label='{} '.format(var_sn)+'[W/m^2]')

    # Specify source, plot
    cds = ColumnDataSource(df_subset)
    plt_scatter = subplot.circle(x=var_cnr, y=var_sn,
                               source=cds,
                               hit_dilation=radius,
                               size=markersize)
    # add hover tool for these data points
    hover = HoverTool(tooltips=tooltips,
                      renderers=[plt_scatter], 
                      formatters={"@time": "datetime"},
                      point_policy='snap_to_data',
                      mode='mouse')
    subplot.add_tools(hover)
    
    # set x, y-limits
    subplot.x_range = Range1d(df_subset.min().min() - 10,
                              df_subset.max().max() + 10)
    subplot.y_range = Range1d(df_subset.min().min() - 10,
                              df_subset.max().max() + 10)
    
    # add 1:1 line to guide the eye
    subplot.line(x=[-200, 1500], y=[-200, 1500],
                 line_dash='dashed', line_color='black', line_width=2)
    
    
    return subplot

In [ ]:
# Plot timeseries, but include an extra variable from the weather station

def plot_timeseries_with_tawes(df, var, tawes, tawes_var=None, 
                                           plot_width=500, plot_height=300,
                                           markersize=0.5, radius=0.5,
                                           cut=False, cut_index=150000,
                                           only_midday=False, start_time='11:00', end_time='13:00'):

    # choose a given variable for both instruments from the full data set
    var_cnr = var + '_CNR4'
    var_sn = var + '_SN500'
    
    # subset of the full dataset with only the desired variables
    df_subset = df[[var_cnr, var_sn]].copy()
    if tawes_var is not None:
        df_subset[tawes_var] = tawes[tawes_var]
    
    
    # cut first, then select midday values 
    # (if both "cut" and "only_noon" are True, we want to exclude the noon times past the cut_index as well)
    if cut:
        # select only data up to a given index
        df_subset = df_subset[:cut_index]
    if only_midday:
        # choose time +/- 1 hour around noon or whatever times are set
        df_subset = df_subset.between_time(start_time, end_time)
    
    # FIGURE INITIATION
    # initiate subplot_series for a given variable: timeseries
    subplot_series = figure(tools = 'box_zoom, pan, reset, save',
                            title='Time series with a TAWES variable ({})'.format(tawes_var),
                            plot_width=plot_width, 
                            plot_height=plot_height,
                            x_axis_type='datetime',
                            x_axis_label='Time',
                            y_axis_label='{} '.format(var)+'[W/m^2]')
    
    # add a twin x-axis for a TAWES variable if one is given
    if tawes_var is not None:
        # define extra axis
        subplot_series.extra_y_ranges = {'tawes_var': Range1d(start=df_subset[tawes_var].min(),
                                                              end=df_subset[tawes_var].max())}
        # add it to figure
        subplot_series.add_layout(LinearAxis(y_range_name='tawes_var'), "right")

    
    # NET RADIOMETER PLOTS
    # Add points at times when CNR4 > SN500
    cds_1 = ColumnDataSource(df_subset[df_subset[var_cnr] > df_subset[var_sn]])
    plt_cnr_1 = subplot_series.circle(x='time', y=var_cnr,
                                      source=cds_1,
                                      hit_dilation=radius,
                                      color='navy',
                                      size=markersize)
    plt_sn_1 = subplot_series.circle(x='time', y=var_sn,
                                     source=cds_1,
                                     hit_dilation=radius,
                                     color='darkcyan',
                                     size=markersize)
    if tawes_var is not None:
        # plot the TAWES variable on the extra axis
        tawes_plot_1 = subplot_series.circle(x='time',
                                           y=tawes_var,
                                           source=cds_1,
                                           size=0.5*markersize,
                                           y_range_name='tawes_var',
                                           level='underlay',
                                           color="green")

    # Add points at times when CNR4 < SN500
    cds_2 = ColumnDataSource(df_subset[df_subset[var_cnr] < df_subset[var_sn]])
    plt_sn_2 = subplot_series.circle(x='time', y=var_sn,
                                     source=cds_2,
                                     hit_dilation=radius,
                                     color='maroon',
                                     size=0.5*markersize)
    plt_cnr_2 = subplot_series.circle(x='time', y=var_cnr,
                                      source=cds_2,
                                      hit_dilation=radius,
                                      color='darkorange',
                                      size=markersize)
    if tawes_var is not None:
        # plot the TAWES variable on the extra axis
        tawes_plot_2 = subplot_series.circle(x='time',
                                           y=tawes_var,
                                           source=cds_2,
                                           size=markersize,
                                           y_range_name='tawes_var',
                                           level='underlay',
                                           color="green")

    # set x, y-limits
    subplot_series.x_range = Range1d(df_subset.index[0], df_subset.index[-1])
    subplot_series.y_range = Range1d(df_subset[[var_cnr, var_sn]].min().min(),
                                     df_subset[[var_cnr, var_sn]].max().max())
    
    # horizontal line to guide the eye at 0
    hline = Span(location=0, dimension='width',
                 line_dash='dashed', line_color='black', line_width=1)
    subplot_series.add_layout(hline)
    
    # LEGEND AND HOVERTOOLS
    # legend items and tooltips for the hover tools
    if tawes_var is not None:
        # add legend items + hover tools WITH the TAWES variable
        legend_items = [('SN500 (SN500 > CNR4)', [plt_sn_2]),
                        ('CNR4 (SN500 > CNR4)', [plt_cnr_2]),
                        ('CNR4 (CNR4 > SN500)', [plt_cnr_1]),
                        ('SN500 (CNR4 > SN500)', [plt_sn_1]),
                        ('Right axis: TAWES ({})'.format(tawes_var), [tawes_plot_1])]
        tooltips_series = [('Time', '@time{%Y-%m-%d %H:%M}'), 
                           ('CNR4', '@{}'.format(var_cnr)),
                           ('SN500', '@{}'.format(var_sn)),
                           ('TAWES ({})'.format(tawes_var), '@{}'.format(tawes_var))]

    else:
        # add legend items + hover tools WITHOUT the TAWES variable
        legend_items = [('SN500 (SN500 > CNR4)', [plt_sn_2]),
                        ('CNR4 (SN500 > CNR4)', [plt_cnr_2]),
                        ('CNR4 (CNR4 > SN500)', [plt_cnr_1]),
                        ('SN500 (CNR4 > SN500)', [plt_sn_1])]
        tooltips_series = [('Time', '@time{%Y-%m-%d %H:%M}'), 
                           ('CNR4', '@{}'.format(var_cnr)),
                           ('SN500', '@{}'.format(var_sn))]
    
    # add legend with the selected items
    legend = Legend(items=legend_items,
                    location=(0, 20),
                    orientation='vertical')
    subplot_series.add_layout(legend, 'below')
    
    # define hover tools for all the data points
    h_1 = HoverTool(tooltips=tooltips_series,
                    renderers=[plt_cnr_1],
                    formatters={"@time": "datetime"},
                    mode='vline')
    h_2 = HoverTool(tooltips=tooltips_series,
                    renderers=[plt_sn_2],
                    formatters={"@time": "datetime"},
                    mode='vline')
    # add hover tools to plots
    subplot_series.add_tools(h_1)
    subplot_series.add_tools(h_2)
    
    return subplot_series

In [ ]:
# Plot the ratio+difference between the instruments, given measurements from the weather station at that time

def plot_tawes_vs_radiometers(df, var, tawes, tawes_var=None, 
                              plot_width=450, plot_height=450,
                              markersize=0.5, radius=2,
                              cut=False, cut_index=150000,
                              only_midday=False, start_time='11:00', end_time='13:00'):

    # choose a given variable for both instruments from the full data set
    var_cnr = var + '_CNR4'
    var_sn = var + '_SN500'
    
    # subset of the full dataset with only the desired variables
    df_subset = df[[var_cnr, var_sn]].copy()
    df_subset[tawes_var] = tawes[tawes_var]
    
    # add difference and ratio
    df_subset['SN_minus_CNR'] = df_subset[var_sn] - df_subset[var_cnr]
    df_subset['SN_over_CNR'] = df_subset[var_sn] / df_subset[var_cnr]
    
    # cut first, then select midday values 
    # (if both "cut" and "only_noon" are True, we want to exclude the noon times past the cut_index as well)
    if cut:
        # select only data up to a given index
        df_subset = df_subset[:cut_index]
    if only_midday:
        # choose time +/- 1 hour around noon or whatever times are set
        df_subset = df_subset.between_time(start_time, end_time)
        
    # define data source
    cds = ColumnDataSource(df_subset)
    
    # initiate subplots
    subplot_1 = figure(tools = 'box_zoom, pan, reset, save',
                            title='{} vs. (SN500 - CNR4): {}'.format(tawes_var, var),
                            plot_width=plot_width, 
                            plot_height=plot_height,
                            x_axis_label='SN500 - CNR4 [W/m^2]',
                            y_axis_label='TAWES: {}'.format(tawes_var))
    subplot_2 = figure(tools = 'box_zoom, pan, reset, save',
                            title='{} vs. (SN500 / CNR4): {}'.format(tawes_var, var),
                            plot_width=plot_width, 
                            plot_height=plot_height,
                            x_axis_label='SN500 / CNR4',
                            y_axis_label='TAWES: {}'.format(tawes_var))

    # Plot the TAWES variable vs. difference and ratio
    plt_diff = subplot_1.circle(x='SN_minus_CNR', 
                                y=tawes_var,
                                source=cds,
                                hit_dilation=radius,
                                size=markersize)
    plt_ratio = subplot_2.circle(x='SN_over_CNR', 
                                 y=tawes_var,
                                 source=cds,
                                 hit_dilation=radius,
                                 size=markersize)
    
    # vertical lines to guide the eye at 0/1
    vline_at_0 = Span(location=0, dimension='height',
                   line_dash='dashed', line_color='black', line_width=1)
    vline_at_1 = Span(location=1, dimension='height',
                   line_dash='dashed', line_color='black', line_width=1)
    subplot_1.add_layout(vline_at_0)
    subplot_2.add_layout(vline_at_1)
    
    # tooltips for the hover tools
    tooltips_1 = [('Time', '@time{%Y-%m-%d %H:%M}'), 
                  ('CNR4', '@{}'.format(var_cnr)),
                  ('SN500', '@{}'.format(var_sn)),
                  ('SN500 - CNR4 [W/m^2]', '@SN_minus_CNR'),
                  ('TAWES ({})'.format(tawes_var), '@{}'.format(tawes_var))]
    tooltips_2 = [('Time', '@time{%Y-%m-%d %H:%M}'),
                  ('CNR4', '@{}'.format(var_cnr)),
                  ('SN500', '@{}'.format(var_sn)),
                  ('SN500 / CNR4', '@SN_over_CNR'),
                  ('TAWES ({})'.format(tawes_var), '@{}'.format(tawes_var))]
    # initiate hover tools
    h_1 = HoverTool(tooltips=tooltips_1,
                    renderers=[plt_diff],
                    formatters={"@time": "datetime"},
                    mode='mouse')
    h_2 = HoverTool(tooltips=tooltips_2,
                    renderers=[plt_ratio],
                    formatters={"@time": "datetime"},
                    mode='mouse')
    # add hover tools to plots
    subplot_1.add_tools(h_1)
    subplot_2.add_tools(h_2)
    
    return subplot_1, subplot_2

In [ ]:
# Make a combined interactive plot for a given variable from the radiometers

def make_interactive_plots(df, var,
                           cut=True, cut_index=150000,
                           only_midday=False, start_time='11:00', end_time='13:00'):
    # simple time series
    subplot_series = plot_full_timeseries(df, var, plot_width=900, plot_height=350, radius=1,
                                          cut=cut, cut_index=cut_index,
                                          only_midday=only_midday, start_time=start_time, end_time=end_time)
    # ratio of values from both instruments at all times
    subplot_ratio_series = plot_ratio_timeseries(df, var, plot_width=900, plot_height=250, radius=1,
                                                 cut=cut, cut_index=cut_index,
                                                 only_midday=only_midday, start_time=start_time, end_time=end_time)
    # SN500 vs CNR4: 1:1 scatter plot
    subplot_1_to_1 = plot_one_to_one(df, var, plot_width=450, plot_height=400, radius=4,
                                     cut=cut, cut_index=cut_index,
                                     only_midday=only_midday, start_time=start_time, end_time=end_time)
    # diurnal cycle of median values, indicating also ratio of medians
    subplot_cycle = plot_diurnal_cycle(df, var, plot_width=450, plot_height=350, line_width=1,
                                       cut=cut, cut_index=cut_index,
                                       only_midday=only_midday, start_time=start_time, end_time=end_time)

    # arrange subplots
    show(column([subplot_series, subplot_ratio_series]))
    show(row([subplot_1_to_1, subplot_cycle]))

In [ ]:
# Make a combined interactive plot for a given variable from the radiometers and one weather station variable

def make_interactive_plots_tawes(df, var, tawes, tawes_var=None,
                                 cut=True, cut_index=280000,
                                 only_midday=True, start_time='04:00', end_time='15:00'):
    # make a timeseries plot including the chosen TAWES variable
    tawes_timeseries = plot_timeseries_with_tawes(df, var, 
                                                  tawes, tawes_var=tawes_var, 
                                                  plot_width=900, plot_height=450, 
                                                  radius=1,
                                                  cut=cut, cut_index=cut_index,
                                                  only_midday=only_midday,
                                                  start_time=start_time,
                                                  end_time=end_time)

    # plot TAWES variable vs. the difference and ratio between SN500 and CNR4
    tawes_diff, tawes_ratio = plot_tawes_vs_radiometers(df, var, 
                                       tawes, tawes_var=tawes_var, 
                                       plot_width=450, plot_height=450, radius=3,
                                       cut=cut, cut_index=cut_index,
                                       only_midday=only_midday, start_time=start_time, end_time=end_time)
    # arrange subplots
    show(tawes_timeseries)
    show(row([tawes_ratio, tawes_diff]))

# First look at the data: non-interactive plots for all variables

In [ ]:
plot_full_timeseries_noninteractive(df, 'SWin', cut=cut);
plot_full_timeseries_noninteractive(df, 'SWout', cut=cut);
plot_full_timeseries_noninteractive(df, 'LWin', cut=cut);
plot_full_timeseries_noninteractive(df, 'LWout', cut=cut);

Non-interactive plot: full time series.

+ When looking at the SW data, for the first ∼3 months the values from SN500 are a bit higher than those from CNR4 (the cut-off is shown as the thin black vertical line - I chose this line arbitrarily where it seemed reasonable). For this time period, most data is shown in red/orange colors (which indicates times when SN500 > CNR4). There seems to be only a small systematic difference between the two radiometers - red points lie closely above the orange ones. 
+ After this time period, the SN500 values drop way below those of CNR4 (these times are shown in blue/cyan).  The discrepancy between the two radiometers is not systematic and small anymore.
+ The vertical lines indicate the "cut-off" which is used later: only data points before this point in time are considered if the "cut" flag is set to True.
+ Even before the cut-off line, there are a few times in LWin when CNR4 << SN500 - we'll look at these in more detail later
+ For LWout there's a small systematic offset.

In [ ]:
plot_one_to_one_noninteractive(df, 'SWin', 
                               cut=cut, cut_index=cut_index,
                               only_midday=only_midday, start_time=start_time, end_time=end_time);

plot_one_to_one_noninteractive(df, 'SWout', 
                               cut=cut, cut_index=cut_index,
                               only_midday=only_midday, start_time=start_time, end_time=end_time);

plot_one_to_one_noninteractive(df, 'LWin', 
                               cut=cut, cut_index=cut_index,
                               only_midday=only_midday, start_time=start_time, end_time=end_time);

plot_one_to_one_noninteractive(df, 'LWout', 
                               cut=cut, cut_index=cut_index,
                               only_midday=only_midday, start_time=start_time, end_time=end_time);

These plots compare the values measured by each instrument as a scatter plot, the points are colored by the time of day. The interactive (but non-colored) versions of the plots are shown later in the notebook.

+ For both SW components and for LWout, there’s a clear systematic error. Many points lie clustered along a line slightly above the 1:1 line, i.e. SN500 > CNR4.

+ For both SW components there’s a large (and seemingly random) spread of the points that lie below the 1:1 line. These points are the reason why in the median plot (only interactive, see below) we see that CNR4 > SN500, although the systematic error would indicate the exact opposite (except for LWout).

+ Despite cutting off some of the data, there are clearly non-random outliers in SWin and LWin components. There are "lines" of points that belong together, when one of the instruments show some weird behaviour. Based on the color, for LWin this obviously happens mostly in the morning hours - the exact times when this happens can be checked in the interactive plots.

# Interactive plots

These plots contain quite a few data points, and so it can take a bit of time to produce them, especially if both "cut" and "only_midday" flags are set to False (i.e. all data points are used). 

In the interactive plots I only concentrate on "cut" data, however, the plots can be easily changed by simply setting the "cut" flag to False.

## Incoming shortwave radiation

In [ ]:
make_interactive_plots(df, 'SWin', 
                       cut=cut, cut_index=cut_index,
                       only_midday=True, start_time=start_time, end_time=end_time)

+ For SWin, I don't see anything particularly interesting going on in the simple time series (first plot), regardless of whether we check values for the whole day or only midday (only_midday=True/False). 

+ The red points in the SN500/CNR4 ratio (second plot) indicate times when CNR4 (denominator) values are small - the red points are all over the place because of this. However, if the only_midday flag is set to True, all these points disappear and only the blue ones are left.
    - With only_midday=True, there are multiple "interesting" days when the ratios are not around 1. Only on 03.10. we see SN500 >> CNR4
    - There are multiple days when SN500 << CNR4 at midday: 31.08., 02.09., 09.09, 20.09., 21.09. Links to the webcam pictures for all these dates are below.
    
+ These are also the points that show up as outliers ("trails of data points") in the 1:1 scatter plot (third plot). However, if we ignore these points far below the 1:1 line, we see a systematic offset between the rest of the SN500 and CNR4 values.

+ Because of these data points, the diurnal cycle of medians shows that CNR4 > SN500, which does not correspond to the systematic offset seen in the third plot. The hover tool in the last (fourth) plot also shows the ratio of medians.

+ 03.10.2020 (SN500 > CNR4): rain around midday, on this day the values for both instruments are very low (mostly below 100 W/m^2)  
https://www.foto-webcam.eu/webcam/innsbruck-uni/2020/10/03/1150  
https://www.foto-webcam.eu/webcam/innsbruck-uni/2020/10/03/1230

+ 31.08.2020: cloudy  
https://www.foto-webcam.eu/webcam/innsbruck-uni/2020/08/31/1240

+ 02.09., 20.09., 21.09.: sunny, some clouds  
https://www.foto-webcam.eu/webcam/innsbruck-uni/2020/09/02/1140  
https://www.foto-webcam.eu/webcam/innsbruck-uni/2020/09/20/1210  
https://www.foto-webcam.eu/webcam/innsbruck-uni/2020/09/21/1130

+ 09.09.: sunny, no clouds  
https://www.foto-webcam.eu/webcam/innsbruck-uni/2020/09/09/1200

## Outgoing shortwave radiation

In [ ]:
make_interactive_plots(df, 'SWout', 
                       cut=cut, cut_index=cut_index,
                       only_midday=True, start_time=start_time, end_time=end_time)

+ If only_midday=False, one can see that there's a clear diurnal cycle in the ratios (first and second figure): at midday, SN500 > CNR4 on most days, and SN500 < CNR4 in early morning, late afternoon and at night.

+ In the 1:1 scatter plot, there are even more points below the 1:1 line than for SW in: these come from dates between 12.09.-17.09. and 01.10.-04.10. There are also quite a few times when the values from SN500 are zero even at midday.

+ Same as before for SWin, the median is "distorted" by these outliers.

12.09.-17.09.: no rain at all around midday, some clouds / no clouds  
https://www.foto-webcam.eu/webcam/innsbruck-uni/2020/09/13/1210  
https://www.foto-webcam.eu/webcam/innsbruck-uni/2020/09/15/1210

01.10.-04.10.: few clouds on Oct 1st, cloudy on Oct 2nd, rain on Oct 3rd (no data), sun+some clouds again oon Oct 4th  
https://www.foto-webcam.eu/webcam/innsbruck-uni/2020/10/01/1200  
https://www.foto-webcam.eu/webcam/innsbruck-uni/2020/10/02/1140  
https://www.foto-webcam.eu/webcam/innsbruck-uni/2020/10/03/1200

## Incoming longwave radiation

In [ ]:
make_interactive_plots(df, 'LWin', 
                       cut=True, cut_index=cut_index,
                       only_midday=False, start_time=start_time, end_time=end_time)

+ In the time series plots there are a few interesting times when SN500 >> CNR4. Most "well defined" peaks can be easily identified with the plot of the ratios. These peaks occur on:
    - 22.07. around 13:00 - 14:00
    - 09.09. and 19.09. before sunrise (~05:30 - 7:30; zoom in on these peaks in the "simple time series" plot, these are very non-random looking)
    - 17.09. before 11 am
    - 12.10 but this is a bit of a special day: around midday we get both situations when SN500 >> CNR4 and SN500 << CNR4
    - 14.10. between 05-11 am  

+ All these times can be seen as "hairs"/"paths" of the data on the 1:1 scatter plot, when the SN500 values increase within a short time and around 2 hours later they drop down again. There are more than just the ones mentioned below (e.g. 10.10.)

+ 22.07., 13:00-14:00: cloudy + rainy (but no raindrops on the webcam)  
https://www.foto-webcam.eu/webcam/innsbruck-uni/2020/07/22/1330

+ 09.09., ~5:30-sunrise: very few low clouds, clear sky otherwise (and a really nice sunrise!)  
https://www.foto-webcam.eu/webcam/innsbruck-uni/2020/09/09/0620

+ 19.09., ~5:30-sunrise: first thin haze, then mostly clear sky with few thin low clouds  
https://www.foto-webcam.eu/webcam/innsbruck-uni/2020/09/19/0550

+ 17.09.: rainy morning, around 11:00 it starts clearing out  
https://www.foto-webcam.eu/webcam/innsbruck-uni/2020/09/17/1030

+ 12.10.: rainy morning (raindrop on webcam at 11:00), it starts clearing up around 12:00  
https://www.foto-webcam.eu/webcam/innsbruck-uni/2020/10/12/1100  
https://www.foto-webcam.eu/webcam/innsbruck-uni/2020/10/12/1210

+ 14.10.: almost clear sky  
https://www.foto-webcam.eu/webcam/innsbruck-uni/2020/10/14/1000

## Outgoing longwave radiation

In [ ]:
make_interactive_plots(df, 'LWout', 
                       cut=True, cut_index=cut_index,
                       only_midday=False, start_time=start_time, end_time=end_time)

This one is pretty straightforward: there are no erroneous/weird data points, there is only a simple systematic offset of SN500 values being always higher than CNR4.

**Based on the webcam images there are no obvious conditions when the SN500 misbehaves.**

# Winter peak in SWout values

In [ ]:
# a look at the unrealistic-looking peak which can be seen in SWout
# Look only at day-time
make_interactive_plots(df[200000:320000], 'SWout', 
                       cut=False, cut_index=cut_index,
                       only_midday=True, start_time='08:00', end_time='18:00')

+ SN500 gives very low values (during the day, between 08:00 and 18:00) - almost always lower than CNR4, but mostly zero.

+ 03.12.: it snows in the early morning, (webcam covered by snow), afternoon sunny  
https://www.foto-webcam.eu/webcam/innsbruck-uni/2020/12/03/1050  
https://www.foto-webcam.eu/webcam/innsbruck-uni/2020/12/03/1240

+ 26.12.: very thin snow cover again (no snow on the roofs around the day before, 2 days after)  
https://www.foto-webcam.eu/webcam/innsbruck-uni/2020/12/26/1240

+ A bit of new snow seen of the roofs on 29.12., 06.01., 13.01. (and later on 11.02.), corresponding to the peaks in CNR4

+ Major snowfall starting on 14.01.: webcam completely covered until 16.01. midday  
https://www.foto-webcam.eu/webcam/innsbruck-uni/2021/01/14/1040  

+ At least some snow persists until 27.12., until th 28th it snows again and by 01.02. most of the snow in the city disappears  
https://www.foto-webcam.eu/webcam/innsbruck-uni/2021/01/27/1320  
https://www.foto-webcam.eu/webcam/innsbruck-uni/2021/01/28/1320  
https://www.foto-webcam.eu/webcam/innsbruck-uni/2021/02/01/1320

**The peaks in SWout correspond to the periods with snow, however, there's a big difference between the two instruments during these times.**

# TAWES data

TAWES Data obtained from https://acinn-data.uibk.ac.at/station/1/S201708/  
This data set does **NOT** contain data after 20.01.2021 (i.e. almost two months are missing.), and there are some other gaps in the data.

+ **tl2**: temperature, university rooftop [C] 
+ **rf2**: relative humidity, university rooftop [%]
+ **rr**: Precipitationwaage weightbridge, basevalue [units should be mm, but it doesn't seem to be the case]
+ **rrm**: Precipitation, basevalue [no units given]
+ **som**: sunshine duration [s]
+ **glom**: Global radiation, basevalue [mV]

I looked mostly at **temperature** data (tl2), but I decided to leave the other variables in there just in case someone wants to play around a bit, looking at different combinations of radiometer/TAWES variables. 

In [ ]:
# load data
tawes = pd.read_csv('tawes_data.csv', sep=';', skiprows=[0], index_col=0, 
                    parse_dates=True, infer_datetime_format=True)

# select some data variables
tawes = tawes[['tl2', 'rf2', 'rr', 'rrm', 'som', 'glom']]
tawes

## Temperature vs. SWin, SWout

In [ ]:
# TAWES data not available after 20.01., corresponding to index ~280000
make_interactive_plots_tawes(df, 'SWin', tawes, tawes_var='tl2',
                             cut=True, cut_index=280000,
                             only_midday=True, start_time='10:00', end_time='15:00')

**Note the scale on the x-axis of the right plot: zoom in.**  


In [ ]:
# TAWES data not available after 20.01., corresponding to index ~280000
make_interactive_plots_tawes(df, 'SWout', tawes, tawes_var='tl2',
                             cut=True, cut_index=280000,
                             only_midday=True, start_time='10:00', end_time='15:00')

I only considered daytime values here (10:00-15:00, see code blocks).
More discrepancies between the datasets seem to occur at lower temperatures, however, not exclusively (see e.g. SWin on 20.09., at around 25°C on a sunny day).
However, interestingly, the ratio and the difference between the instruments seem to be weakly temperature-dependent.

## Temperature vs. LWin, LWout

Here I am interested also in the weird spikes in data which occur before sunrise, and so I look at a more extended time period during the day.

In [ ]:
make_interactive_plots_tawes(df, 'LWin', tawes, tawes_var='tl2',
                             cut=True, cut_index=280000,
                             only_midday=True, start_time='04:00', end_time='18:00')

Again, more spikes seem to occur at lower temperatures (more "hair" in the plot when temperature is closer to zero).
However, when zooming in on the spikes in the time series (e.g. 09.09., 19.09.), nothing specific is going on there with temperature - it seems that temperature is not the cause for those spikes. 
But again, the difference between the instruments seems to systematically depend on temperature.

In [ ]:
make_interactive_plots_tawes(df, 'LWout', tawes, tawes_var='tl2',
                             cut=True, cut_index=280000,
                             only_midday=True, start_time='04:00', end_time='18:00')

At least there's nothing weird going on with LWout!

# Heating 

**Heating data is unfortunately not available.**  
...and so I could obviously not take a look at that.